<a href="https://colab.research.google.com/github/achaudhury7378/Online-Learning-Project-/blob/main/HyperBand_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu
!pip install keras-tuner

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from keras.losses import categorical_crossentropy

from kerastuner.tuners import Hyperband

In [ ]:
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11493376/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

batch_size = 128
epochs = 12

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
469/469 [==============================] - 8s 16ms/step - loss: 0.4707 - accuracy: 0.8499 - val_loss: 0.0516 - val_accuracy: 0.9843
Epoch 2/12
469/469 [==============================] - 7s 15ms/step - loss: 0.0904 - accuracy: 0.9739 - val_loss: 0.0364 - val_accuracy: 0.9879
Epoch 3/12
469/469 [==============================] - 7s 15ms/step - loss: 0.0639 - accuracy: 0.9807 - val_loss: 0.0297 - val_accuracy: 0.9892
Epoch 4/12
469/469 [==============================] - 7s 15ms/step - loss: 0.0528 - accuracy: 0.9838 - val_loss: 0.0320 - val_accuracy: 0.9896
Epoch 5/12
469/469 [==============================] - 7s 15ms/step - loss: 0.0427 - accuracy: 0.9868 - val_loss: 0.0277 - val_accuracy: 0.9912
Epoch 6/12
469/469 [==============================] - 7s 15ms/step - loss: 0.0378 - accuracy: 0.9882 - val_loss: 0.0309 - val_accuracy: 0.9895
Epoch 7/12
469/469 [==============================] - 7s 15ms/step - loss: 0.0342 - accuracy: 0.9891 - val_loss: 0.0301 - val_accuracy: 0.9909

In [ ]:
def MNISTHyperModel(hp):
    num_classes = 10
    model = keras.Sequential()
    
    model.add(Conv2D(
            filters=hp.Choice('num_filters_1', values=[16, 32, 64], default = 32),
            kernel_size=hp.Choice('kernel_1', values=[2,3,4,5], default = 3),
            strides = hp.Choice('stride_1', values=[1,2,3], default = 1),
            activation = 'relu',
            input_shape=(28,28,1)))
    
    model.add(Conv2D(
            filters=hp.Choice('num_filters_2', values=[16, 32, 64], default = 32),
            kernel_size=hp.Choice('kernel_2', values=[2,3,4,5], default = 3),
            strides = hp.Choice('stride_2', values=[1,2,3], default = 1),
            activation='relu'))
    
    model.add(MaxPooling2D(pool_size=2))
    
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.0, max_value=0.5, default=0.3, step=0.1)))
    
    model.add(Flatten())
    
    model.add(Dense(
            units=hp.Int('dense_units', min_value=32, max_value=512, step=32, default=128),
            activation=hp.Choice('dense_activation', values=['relu', 'tanh', 'sigmoid'], default='relu')))
    
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.0, max_value=0.5, default=0.5, step=0.1)))                   
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(
        optimizer=Adam(
            hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG', default=1e-3)),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
tuner = Hyperband(
    MNISTHyperModel,
    max_epochs=12,
    objective='val_accuracy',
    executions_per_trial=3,
    directory='hyperband',
    project_name='MNIST_hyperband_2')

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 11
num_filters_1 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64], 'ordered': True}
kernel_1 (Choice)
{'default': 3, 'conditions': [], 'values': [2, 3, 4, 5], 'ordered': True}
stride_1 (Choice)
{'default': 1, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
num_filters_2 (Choice)
{'default': 32, 'conditions': [], 'values': [16, 32, 64], 'ordered': True}
kernel_2 (Choice)
{'default': 3, 'conditions': [], 'values': [2, 3, 4, 5], 'ordered': True}
stride_2 (Choice)
{'default': 1, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
dropout_1 (Float)
{'default': 0.3, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
dense_units (Int)
{'default': 128, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
dense_activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
dropout_2 (Float)
{'default': 0.

In [ ]:
tuner.search(x_train, y_train, validation_split=0.1)

Trial 30 Complete [00h 06m 11s]
val_accuracy: 0.9839999874432882

Best val_accuracy So Far: 0.99272221326828
Total elapsed time: 01h 06m 18s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in hyperband/MNIST_hyperband_2
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_filters_1: 32
kernel_1: 5
stride_1: 1
num_filters_2: 64
kernel_2: 4
stride_2: 1
dropout_1: 0.1
dense_units: 352
dense_activation: relu
dropout_2: 0.4
learning_rate: 0.00014682754255180837
tuner/epochs: 12
tuner/initial_epoch: 4
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 3cf4f267930e71e9c0e8402a52dc0441
Score: 0.99272221326828
Trial summary
Hyperparameters:
num_filters_1: 32
kernel_1: 5
stride_1: 3
num_filters_2: 64
kernel_2: 4
stride_2: 1
dropout_1: 0.30000000000000004
dense_units: 160
dense_activation: relu
dropout_2: 0.30000000000000004
learning_rate: 0.0009259685801345267
tuner/epochs: 12
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.9915000001589457
Trial summary
Hyperparameters:
num_filters_1: 32
kernel_1: 3
stride_1: 1
num_filters_2: 16
kernel_2: 3
stride_2: 2
dropout_1: 0.0
dense_units: 384
dense_a

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
loss, accuracy = best_model.evaluate(x_test, y_test)
print(accuracy)

313/313 [==============================] - 2s 4ms/step - loss: 0.0255 - accuracy: 0.9899
0.9927999973297119


In [ ]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 64)        32832     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 10, 10, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6400)              0         
_________________________________________________________________
dense (Dense)                (None, 352)               2253152   
_________________________________________________________________
dropout_1 (Dropout)          (None, 352)               0